<a href="https://colab.research.google.com/github/abhijitdarekar/Machine_Learning/blob/main/Fine%20Tuning%20HuggingFace%20Model%20with%20PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gc
gc.collect(2)

23

In [9]:
!pip -q install datasets transformers  bitsandbytes  accelerate peft evaluate sentencepiece wandb
# !pip install datasets transformers sentencepiece accelerate peft bitsandbytes evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset , get_dataset_config_names
from random import randrange

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments

from peft import TaskType
from peft import LoraConfig, prepare_model_for_int8_training, get_peft_model, AutoPeftModelForCausalLM, PeftConfig
import warnings
import wandb
warnings.filterwarnings("ignore")
import os

In [11]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
DATASET = "opus100"
CHECK_POINT = 'google/flan-t5-base'
os.environ["WANDB_PROJECT"]="peft and lora"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"

dataset_config = get_dataset_config_names('opus100')
print(dataset_config)

['af-en', 'am-en', 'an-en', 'ar-de', 'ar-en', 'ar-fr', 'ar-nl', 'ar-ru', 'ar-zh', 'as-en', 'az-en', 'be-en', 'bg-en', 'bn-en', 'br-en', 'bs-en', 'ca-en', 'cs-en', 'cy-en', 'da-en', 'de-en', 'de-fr', 'de-nl', 'de-ru', 'de-zh', 'dz-en', 'el-en', 'en-eo', 'en-es', 'en-et', 'en-eu', 'en-fa', 'en-fi', 'en-fr', 'en-fy', 'en-ga', 'en-gd', 'en-gl', 'en-gu', 'en-ha', 'en-he', 'en-hi', 'en-hr', 'en-hu', 'en-hy', 'en-id', 'en-ig', 'en-is', 'en-it', 'en-ja', 'en-ka', 'en-kk', 'en-km', 'en-kn', 'en-ko', 'en-ku', 'en-ky', 'en-li', 'en-lt', 'en-lv', 'en-mg', 'en-mk', 'en-ml', 'en-mn', 'en-mr', 'en-ms', 'en-mt', 'en-my', 'en-nb', 'en-ne', 'en-nl', 'en-nn', 'en-no', 'en-oc', 'en-or', 'en-pa', 'en-pl', 'en-ps', 'en-pt', 'en-ro', 'en-ru', 'en-rw', 'en-se', 'en-sh', 'en-si', 'en-sk', 'en-sl', 'en-sq', 'en-sr', 'en-sv', 'en-ta', 'en-te', 'en-tg', 'en-th', 'en-tk', 'en-tr', 'en-tt', 'en-ug', 'en-uk', 'en-ur', 'en-uz', 'en-vi', 'en-wa', 'en-xh', 'en-yi', 'en-yo', 'en-zh', 'en-zu', 'fr-nl', 'fr-ru', 'fr-zh', 

#### Loading Dataset

In [13]:
dataset = load_dataset(DATASET,"en-fr")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [15]:
tokenizer  = AutoTokenizer.from_pretrained(
  CHECK_POINT,
  load_in_8bit=True,
  device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [16]:
sample = list(dataset['train']['translation'][0].values())

In [17]:

print("Sample Tokenized :",tokenizer(sample))

Sample Tokenized : {'input_ids': [[37, 97, 230, 19, 3, 3076, 10, 4018, 3, 5, 1], [37, 97, 230, 19, 3, 3076, 10, 3076, 3, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [18]:
def preprocess(examples):
    en = [data['en'] for data in examples['translation']]
    fr = [data['fr'] for data in examples['translation']]

    inputs = tokenizer(en, truncation=True)
    labels = tokenizer(fr, truncation=True)

    inputs['labels'] = labels['input_ids']
    return inputs

In [19]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [41]:
train_dataset = tokenized_dataset['train'].select(range(50000))

In [42]:
train_dataset

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})

#### Loading Model

In [43]:
from transformers import AutoModelForSeq2SeqLM

import accelerate
model = AutoModelForSeq2SeqLM.from_pretrained(CHECK_POINT, load_in_8bit=True, device_map="auto")


#### Creating a PEFT Model

In [44]:
model = prepare_model_for_int8_training(model)

In [45]:
peft_config = LoraConfig(
    task_type= TaskType.SEQ_2_SEQ_LM,
    r=4,
    lora_alpha=32,
    lora_dropout=0.01,
    target_modules=['k',"q","v"]
)

peft_model = get_peft_model(model,peft_config)

In [46]:
peft_model.print_trainable_parameters()

trainable params: 663,552 || all params: 248,241,408 || trainable%: 0.2673010942638546


### Model Training

In [47]:
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np

In [48]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,\
    model=peft_model,\
)

In [49]:
accuracy = evaluate.load("accuracy")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [50]:
from transformers import Trainer, TrainingArguments

In [51]:
torch.set_default_dtype(torch.float32)

In [55]:
output_dir = "en2fr"
num_epochs = 20
batch_size = 8

trainingArguments = TrainingArguments(output_dir=output_dir,\
                                      auto_find_batch_size=True,\
                                      learning_rate=1e-3,\
                                      num_train_epochs=num_epochs,\
                                      save_strategy="no",\
                                     report_to="wandb")

In [56]:
trainer = Trainer(model = peft_model,\
                  tokenizer=tokenizer,\
                  data_collator=data_collator,\
                  train_dataset=train_dataset
                  ,eval_dataset=tokenized_dataset['validation'],compute_metrics=compute_metrics)

peft_model.config.use_cache = False

In [ ]:
trainer.train()

Step,Training Loss
500,2.614400
1000,2.223700
1500,2.158400
2000,2.180600
2500,2.152300
3000,2.159700
3500,2.142400
4000,2.121800
4500,2.130400
5000,2.111300
